![Banner](images/banner.png)

# Oracle Database Objects and Collections

Documentation reference link: [Fetching Oracle Database Objects and Collections](https://python-oracledb.readthedocs.io/en/latest/user_guide/sql_execution.html#fetching-oracle-database-objects-and-collections)

In [ ]:
import oracledb

In [ ]:
un = "pythondemo"
pw = "welcome"
cs = "localhost/orclpdb1"

connection = oracledb.connect(user=un, password=pw, dsn=cs)

# Binding Named Objects

Create a demonstration table. This table uses the predefined SDO_GEOMETRY object which stores spatial information:

In [ ]:
with connection.cursor() as cursor:
    try:
        cursor.execute("drop table TestGeometry")
    except:
        ;
        
    cursor.execute("""create table TestGeometry (
                      IntCol   number(9) not null,
                      Geometry sdo_geometry not null)""")
    
print("Done")

Using python-oracledb functions like `gettype()` and `extend()` you can create a Python representation of the database object:

In [ ]:
with connection.cursor() as cursor:
    
    typeObj = connection.gettype("SDO_GEOMETRY")
    elementInfoTypeObj = connection.gettype("SDO_ELEM_INFO_ARRAY")
    ordinateTypeObj = connection.gettype("SDO_ORDINATE_ARRAY")

    obj = typeObj()               # Alternatively use 'obj = typeObj.newobject()''
    obj.SDO_GTYPE = 2003
    obj.SDO_ELEM_INFO = elementInfoTypeObj()
    obj.SDO_ELEM_INFO.extend([1, 1003, 3])
    obj.SDO_ORDINATES = ordinateTypeObj()
    obj.SDO_ORDINATES.extend([1, 1, 5, 7])

Calling `gettype()` requires multiple round-trips to the database, so avoid calling it unnecessarily.

The new object can be bound directly for insertion:

In [ ]:
with connection.cursor() as cursor:
    cursor.execute("insert into TestGeometry values (1, :objbv)",  {"objbv": obj})
    
print("Done")

And then fetched back:

In [ ]:
with connection.cursor() as cursor:
    for (id, obj) in cursor.execute("select IntCol, Geometry from testgeometry"):
        print(id, obj)

Simple attribute access is easy:

In [ ]:
with connection.cursor() as cursor:
    for (id, obj) in cursor.execute("select IntCol, Geometry from testgeometry"):
        print("SDO_GTYPE is", obj.SDO_GTYPE)

 To display all attributes, create a helper function:

In [ ]:
# Oracle Database object dumper

def dumpobject(obj, prefix = "  "):
    if obj.type.iscollection:
        print(prefix, "[")
        for value in obj.aslist():
            if isinstance(value, oracledb.Object):
                dumpobject(value, prefix + "  ")
            else:
                print(prefix + "  ", repr(value))
        print(prefix, "]")
    else:
        print(prefix, "{")
        for attr in obj.type.attributes:
            value = getattr(obj, attr.name)
            if isinstance(value, oracledb.Object):
                print(prefix + "  " + attr.name + " :")
                dumpobject(value, prefix + "    ")
            else:
                print(prefix + "  " + attr.name + " :", repr(value))
        print(prefix, "}")

Using the helper function shows the full object structure:

In [ ]:
with connection.cursor() as cursor:
    for (id, obj) in cursor.execute("select IntCol, Geometry from testgeometry"):
        print("Id: ", id)
        dumpobject(obj)

# PL/SQL Collections

The sample schema uses PL/SQL collections

In [ ]:
cursor = connection.cursor()

cursor.execute("select dbms_metadata.get_ddl('PACKAGE', 'PKG_DEMO') from dual")
ddl, = cursor.fetchone()
print(ddl.read())

To get a collection, create a Python variable with the database object type:

In [ ]:
typeObj = connection.gettype("PKG_DEMO.UDT_STRINGLIST")
obj = typeObj()

# call the stored procedure which will populate the object
cursor = connection.cursor()
cursor.callproc("pkg_Demo.DemoCollectionOut", (obj,))

To show the collection indexes and values:

In [ ]:
ix = obj.first()
while ix is not None:
    print(ix, "->", obj.getelement(ix))
    ix = obj.next(ix)
print()

Show the values as a simple list:

In [ ]:
print(obj.aslist())

Show the values as a simple dictionary:

In [ ]:
print(obj.asdict())

# Binding PL/SQL Records

Create a new Python object of the correct type and set attribute values:

In [ ]:
import datetime

typeObj = connection.gettype("PKG_DEMO.UDT_DEMORECORD")
obj = typeObj()

obj.NUMBERVALUE = 6
obj.STRINGVALUE = "Test String"
obj.DATEVALUE = datetime.datetime(2016, 5, 28)
obj.BOOLEANVALUE = False

Call the stored procedure which will modify the object:

In [ ]:
with connection.cursor() as cursor:
    cursor.callproc("pkg_Demo.DemoRecordsInOut", (obj,))

Show the modified values:

In [ ]:
print("NUMBERVALUE ->", obj.NUMBERVALUE)
print("STRINGVALUE ->", obj.STRINGVALUE)
print("DATEVALUE ->", obj.DATEVALUE)
print("BOOLEANVALUE ->", obj.BOOLEANVALUE)